## Analytic approximations to the min and max operators: 

## 1. The average relative error as a test for numerical stability: 

In [15]:
using Statistics

function numerical_stability(method,type::Int64)

	relative_errors = zeros(100)

	for i = 1:100

		X = (2*rand(100).-1.0)*1000

		if type == 1

			max_ = maximum(X)

			relative_errors[i] = abs(max_-method(X,i))/abs(max_)

		else 

			min_ = minimum(X)

			relative_errors[i] = abs(min_-method(X,i))/abs(min_)

		end

	end

	return mean(relative_errors)

end

numerical_stability (generic function with 1 method)

## 2. A couple answers proposed on the MathOverflow: https://mathoverflow.net/questions/35191/a-differentiable-approximation-to-the-minimum-function


In [2]:
function GM(X::Array{Float64, 1},N::Int64)

	return mean(X.^N)^(1/N)

end

function exp_GM(X::Array{Float64, 1},N::Int64)

	### This method is terrible. Overflow errors everywhere. 

	exp_ = mean(exp.(N*X))

	return (1/N)*log(exp_)

end

exp_GM (generic function with 1 method)

## 3. Analysis of the generalised mean: 

In [3]:
numerical_stability(GM,1)

DomainError: DomainError with -1.6737682885546803e13:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

### This method returns a type error for odd exponents unless all elements of X are positive: https://math.stackexchange.com/questions/317528/how-do-you-compute-negative-numbers-to-fractional-powers/317546#317546.  

In [6]:
X = 100*(2*rand(100).-1.0)

GM(X,101), maximum(X)

DomainError: DomainError with -6.321674060509639e199:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

### It is also numerically unstable:

In [7]:
X = 1000*rand(100)

GM(X,105), maximum(X)

(Inf, 999.0314747417859)

## 4. Analysis of an exponential variant of the generalised mean: 

In [8]:
numerical_stability(exp_GM,1)

Inf

### This approach works for vectors containing small elements:

In [9]:
X = 5*rand(10)

exp_GM(X,10), maximum(X)

(4.67592809202989, 4.8879261224933686)

### But, consider vectors with values in the interval \[0,500\]:

In [10]:
X = 500*rand(10)

exp_GM(X,10), maximum(X)

(Inf, 490.55456844015566)

## 5. Analysis of the smooth maximum: https://en.wikipedia.org/wiki/Smooth_maximum

In [11]:
function smooth_max(X::Array{Float64, 1},N::Int64)

	## implementation of the smooth maximum: 
	## https://en.wikipedia.org/wiki/Smooth_maximum

	exp_ = exp.(X*N)

	return sum(X.*exp_)/sum(exp_)

end

numerical_stability(smooth_max,1)

NaN

### This method is also numerically unstable. 

### Summary: We need a method that handles vectors sampled from large intervals without producing overflow errors, has good convergence properties, and handles both positive and negative numbers. 

## 6. My proposed solutions inspired by the infinity norm: 

In [12]:
## The key ideas here are to first standardise the vector by subtracting the mean
## and dividing by the variance. Then we map the vector from R^n to R+^n so 
## a method inspired by the infinity norm can work. 

function analytic_max(X::Array{Float64, 1},N::Int64)

	mu, sigma = mean(X), std(X)

    Z_score = (X.-mu)./sigma

    exp_sum = sum(exp.(Z_score*N))

    log_ = log(exp_sum)/N

    return (log_*sigma)+mu

end

function analytic_min(X::Array{Float64, 1},N::Int64)

	mu, sigma = mean(X), std(X)

    Z_score = (X.-mu)./sigma

    neg_exp_sum = sum(exp.(-Z_score*N))

    log_ = -log(neg_exp_sum)/N

    return (sigma*log_)+mu

end

analytic_min (generic function with 1 method)

In [13]:
numerical_stability(analytic_max,1)

0.054149551817268105

In [16]:
numerical_stability(analytic_min,2)

0.05143422086633216